In [2]:
%env SPECTRAL_CONNECTIVITY_ENABLE_GPU=true

env: SPECTRAL_CONNECTIVITY_ENABLE_GPU=true


In [3]:
import os
os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')

import spikeinterface.extractors as se 
import cupy as xp
from cupyx.scipy.fft import ifft
from cupyx.scipy.sparse.linalg import svds
from spectral_connectivity import Multitaper, Connectivity
import importlib
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import lfp.lfp_analysis.LFP_collection as LFP_collection
import pickle 
def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl
    Returns:
        none
    """
    with open(file_name, "wb") as file:
        pickle.dump(thing_to_pickle, file)
def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, "rb") as file:
        return pickle.load(file)

df = pd.read_excel(r"lfp/channel_mapping_sme.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}







for _, row in df.iterrows():
    subject = row['Subject'].astype(str)
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
behavior_dicts = {}
#pickle_this(subject_to_channel_dict)

def make_recording_to_subj_dict(data_path):
    recording_to_subject = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                subject = str(int((file.split("_")[0]))/10)
                recording_to_subject[file] = subject
                behavior_dicts[file] = {}
    return recording_to_subject

# def process(data_path):
#     recording_to_subject = make_recording_to_subj_dict(data_path)
#     print(recording_to_subject)
#     collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 4)
#     #collection.process()
#     return collection    
    

    



In [5]:
import lfp.lfp_analysis.LFP_recording as rec 
rec_path = r"data/new_object_control_data/20250618_113931_object_control_subj_1-2_and_6-1.rec/20250618_113931_object_control_subj_6-1_merged.rec"
rec_folder = r"data/new_object_control_data"
# rec_object = rec.LFPRecording(subject = '1.2', channel_dict = {'mPFC': 2,
#                                                                'IC':27,
#                                  'NAc': 28,
#                                  'MD': 29,
#                                  'LH': 30,
#                                  'BLA': 31}, merged_rec_path = rec_path)



Processing 20250618_113931_object_control_subj_6-1_merged.rec
Found first timestamp


In [11]:
channel_dict = {'mPFC': 2,'IC':27,'NAc': 28,
                                 'MD': 29,
                                 'LH': 30,
                                 'BLA': 31}

def make_recording_to_subj_dict_eli(data_path):
    recording_to_subject = {}
    subject_to_channel_dict = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                parts = file.replace('-','_').split('_')
                subject = '.'.join(parts[5:7])
                recording_to_subject[file] = subject
                subject_to_channel_dict[subject] = channel_dict
                # behavior_dicts[file] = {}
    return recording_to_subject

recording_to_subject_dict = make_recording_to_subj_dict_eli(rec_folder)
unpickle_this(r"pilot2/

6.1 <class 'str'>
1.2 <class 'str'>
4.3 <class 'str'>
4.4 <class 'str'>
4.2 <class 'str'>
4.1 <class 'str'>
1.3 <class 'str'>
6.3 <class 'str'>


In [ ]:
LFP_collection.collection(

In [15]:
rec_raw = se.read_spikegadgets(rec_path, stream_id = "trodes")
raw_traces = rec_raw.get_traces()
print(rec_raw)
print(raw_traces.shape)

SpikeGadgetsRecordingExtractor: 32 channels - 1 segments - 20.0kHz - 1939.502s
  file_path: data/new_object_control_data/20250618_113931_object_control_subj_1-2_and_6-1.rec/20250618_113931_object_control_subj_6-1_merged.rec
(38790035, 32)


In [16]:
rec_object.traces.shape

(0, 5)

In [12]:
trodes_rec = rec_object._read_trodes()
traces = trodes_rec.get_traces()

Processing 20250618_113931_object_control_subj_6-1_merged.rec


In [13]:
print(traces.shape)
print(trodes_rec)


(0, 32)
ResampleRecording: 32 channels - 1 segments - 1.0kHz - 1939.501s


In [6]:
traces_only_some_channels = trodes_rec.get_traces(['2','28','29','30','31'], start_frame = rec_object.first_timestamp)
traces_from start = trodes_rec.get_traces(['2','28','29','30','31'], start_frame = 0)

TypeError: list indices must be integers or slices, not list

In [26]:
rec_object.traces.shape

(0, 5)

In [19]:
data_path = r"data/cups_raw"
recording_to_subject = make_recording_to_subj_dict(data_path)


In [22]:
collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 5)


Processing 23_cups_p4_merged.rec
Found first timestamp
Processing 24_cups_p4_merged.rec
Found first timestamp
Processing 11_cups_p4_merged.rec
Found first timestamp
Processing 31_cups_p4_merged.rec
Found first timestamp
Processing 32_cups_p4_merged.rec
Found first timestamp
Processing 21_cups_p4_merged.rec
Found first timestamp
Processing 22_cups_p4_merged.rec
Found first timestamp


In [26]:
for recording in collection.recordings:
    print(recording.name)
    print(recording.first_timestamp)
    print(recording.traces.shape)
    print(recording.rms_traces.shape)

23_cups_p4_merged.rec
3277582
(0, 5)
(0, 5)
24_cups_p4_merged.rec
1906957
(0, 5)
(0, 5)
11_cups_p4_merged.rec
1675517
(0, 5)
(0, 5)
31_cups_p4_merged.rec
3373207
(0, 5)
(0, 5)
32_cups_p4_merged.rec
1661658
(0, 5)
(0, 5)
21_cups_p4_merged.rec
1786386
(0, 5)
(0, 5)
22_cups_p4_merged.rec
1480109
(0, 5)
(0, 5)


In [28]:
collection.calculate_all()

  0%|          | 0/7 [00:02<?, ?it/s]
/blue/npadillacoreano/mcum/conda/envs/lfp_env/lib/python3.11/site-packages/spectral_connectivity/connectivity.py:348: DeprecationWarning: invalid escape sequence '\p'
  """The normalized imaginary component of the cross-spectrum.
/blue/npadillacoreano/mcum/conda/envs/lfp_env/lib/python3.11/site-packages/spectral_connectivity/connectivity.py:997: DeprecationWarning: invalid escape sequence '\p'
  """Find a range of possible delays from the coherence phase.


CompileException: nvrtc: error: invalid value for --gpu-architecture (-arch)


In [6]:
LFP_collection.LFPCollection.save_to_json(collection, r"data/object_control_sd5")